In [1]:
import os
import cv2
import pandas as pd
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import json
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.models import Model
from keras.layers import Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Activation,GlobalAveragePooling2D, Dense, BatchNormalization, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import SGD

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:


# Set the path to your dataset
dataset_path = "/content/drive/MyDrive/train"

# Load the dataset
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_path,
    image_size=(128, 128),  # Resize images to a consistent size
    batch_size=32,          # Number of images per batch
    label_mode='categorical',  # or 'int' for numeric labels
    shuffle=True            # Shuffle the dataset
)

# Check the dataset
for images, labels in dataset.take(1):
    print(images.shape)  # (batch_size, height, width, channels)
    print(labels.shape)  # (batch_size, number_of_classes)


Found 10009 files belonging to 10 classes.
(32, 128, 128, 3)
(32, 10)


In [4]:
import tensorflow as tf

# Set the path to your dataset
dataset_path = "/content/drive/MyDrive/train"

# Load the training dataset (80%)
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_path,
    image_size=(128, 128),        # Resize images
    batch_size=32,                # Number of images per batch
    validation_split=0.2,         # Reserve 20% for validation (or testing)
    subset='training',            # This is the training set
    seed=123,                     # Reproducibility
    shuffle=True                  # Shuffle the dataset
)

# Load the testing dataset (20%)
test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_path,
    image_size=(128, 128),        # Resize images
    batch_size=32,                # Number of images per batch
    validation_split=0.2,         # Reserve 20% for validation (or testing)
    subset='validation',          # This is the validation/test set
    seed=123,                     # Reproducibility
    shuffle=True                  # Shuffle the dataset
)

# Check the datasets
for images, labels in train_dataset.take(1):
    print(f"Train: {images.shape}, {labels.shape}")

for images, labels in test_dataset.take(1):
    print(f"Test: {images.shape}, {labels.shape}")


Found 10009 files belonging to 10 classes.
Using 8008 files for training.
Found 10009 files belonging to 10 classes.
Using 2001 files for validation.
Train: (32, 128, 128, 3), (32,)
Test: (32, 128, 128, 3), (32,)


In [6]:
#Import the required modules
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.applications.vgg19 import VGG19


# Create the base model from the pre-trained model VGG19
base_model = VGG19(weights='imagenet', include_top=False)

# Add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)

# Add a fully-connected layer
x = Dense(1024, activation='relu')(x)

# Add a logistic layer with the number of classes in your dataset
predictions = Dense(10, activation='softmax')(x) # 10 classes

# this is the model we will train
vgg19_model = Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional VGG19 layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
vgg19_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics = ['accuracy'])

In [7]:
#Getting model's summary
vgg19_model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)           │ (None, None, None, 3)       │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv1 (Conv2D)                │ (None, None, None, 64)      │           1,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv2 (Conv2D)                │ (None, None, None, 64)      │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_pool (MaxPooling2D)           │ (None, None, None, 64)      │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv1 (Conv2D)                │ (None, None, None, 128)     │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv2 (Conv2D)                │ (None, None, None, 128)     │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_pool (MaxPooling2D)           │ (None, None, None, 128)     │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv1 (Conv2D)                │ (None, None, None, 256)     │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv2 (Conv2D)                │ (None, None, None, 256)     │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv3 (Conv2D)                │ (None, None, None, 256)     │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv4 (Conv2D)                │ (None, None, None, 256)     │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_pool (MaxPooling2D)           │ (None, None, None, 256)     │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv1 (Conv2D)                │ (None, None, None, 512)     │       1,180,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv2 (Conv2D)                │ (None, None, None, 512)     │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv3 (Conv2D)                │ (None, None, None, 512)     │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv4 (Conv2D)                │ (None, None, None, 512)     │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_pool (MaxPooling2D)           │ (None, None, None, 512)     │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv1 (Conv2D)                │ (None, None, None, 512)     │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv2 (Conv2D)                │ (None, None, None, 512)     │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv3 (Conv2D)                │ (None, None, None, 512)     │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv4 (Conv2D)                │ (None, None, None, 512)     │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼──────────────

 Total params: 20,559,946 (78.43 MB)

 Trainable params: 535,562 (2.04 MB)

 Non-trainable params: 20,024,384 (76.39 MB)

In [8]:
#Specifing epochs & batch size for vggnet16
epochs = 5
batch_size = 32
image_height = 128
image_width = 128

In [9]:
#Creating an object of ImageDataGenerator for augmenting training dataset
train_datagen = ImageDataGenerator(rescale= 1./255,
rotation_range=10,
width_shift_range=0.1,
height_shift_range=0.1,
shear_range=0.1,
zoom_range=0.1,
horizontal_flip=True,
fill_mode='nearest')

#Creating an object of ImageDataGenerator for augmenting test dataset
test_datagen = ImageDataGenerator(rescale= 1./255)

In [11]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define image size and batch size
image_size = (128, 128)   # VGG19 requires 224x224 input
batch_size = 32           # Number of images to process in a batch

# Define the train data generator with augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,           # Rescale pixel values to [0, 1]
    shear_range=0.2,          # Apply random shear transformations
    zoom_range=0.2,           # Apply random zoom
    horizontal_flip=True,      # Randomly flip images horizontally
    validation_split=0.2      # Split 20% of the data for validation
)

# Define the test/validation data generator (only rescale)
test_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

# Set dataset directory path
dataset_path = "/content/drive/MyDrive/train"

# Create train data generator
train_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=image_size,      # Resize all images to 224x224
    batch_size=batch_size,
    class_mode='categorical',    # For multi-class classification
    subset='training',           # Use this for the training data
    shuffle=True                 # Shuffle the data for better training
)

# Create validation data generator
validation_generator = test_datagen.flow_from_directory(
    dataset_path,
    target_size=image_size,      # Resize all images to 224x224
    batch_size=batch_size,
    class_mode='categorical',    # For multi-class classification
    subset='validation',         # Use this for the validation data
    shuffle=False                # No need to shuffle the validation data
)

# Check the generators
print(f"Train Generator: {len(train_generator)} batches")
print(f"Validation Generator: {len(validation_generator)} batches")


Found 8008 images belonging to 10 classes.
Found 2001 images belonging to 10 classes.
Train Generator: 251 batches
Validation Generator: 63 batches


In [12]:
# Define image size and batch size
image_size = (128, 128)   # VGG19 requires 224x224 input
batch_size = 32           # Number of images to process in a batch

# Define the test data generator (only rescale, no augmentation)
test_datagen = ImageDataGenerator(rescale=1./255)  # Rescale pixel values to [0, 1]

# Set dataset directory path for test data
test_dataset_path = "/content/drive/MyDrive/train"

# Create test data generator
test_generator = test_datagen.flow_from_directory(
    test_dataset_path,
    target_size=image_size,      # Resize all images to 128*128
    batch_size=batch_size,
    class_mode='categorical',    # For multi-class classification
    shuffle=False                # Do not shuffle the test data
)

# Check the test generator
print(f"Test Generator: {len(test_generator)} batches")


Found 10009 images belonging to 10 classes.
Test Generator: 313 batches


In [ ]:
vgg19_history = vgg19_model.fit(train_generator, epochs=epochs, validation_data=validation_generator)

Epoch 1/5


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


251/251 ━━━━━━━━━━━━━━━━━━━━ 2499s 10s/step - accuracy: 0.4266 - loss: 1.6817 - val_accuracy: 0.6257 - val_loss: 1.0922
Epoch 2/5
251/251 ━━━━━━━━━━━━━━━━━━━━ 2288s 9s/step - accuracy: 0.6783 - loss: 0.9556 - val_accuracy: 0.6992 - val_loss: 0.8760
Epoch 3/5
143/251 ━━━━━━━━━━━━━━━━━━━━ 13:05 7s/step - accuracy: 0.7183 - loss: 0.8141

In [1]:
from sklearn.metrics import classification_report

# predictions from the test data
y_pred = vgg19_model.predict(test_generator)

# transform class labels of the predictions
y_pred_classes = np.argmax(y_pred, axis=1)

# Real classes
y_true = test_generator.classes

# Create classification report
report = classification_report(y_true, y_pred_classes, target_names=test_generator.class_indices.keys())

# print report
print(report)

NameError: name 'vgg19_model' is not defined

In [ ]:
vgg19_model.save("/content/drive/MyDrive/tomato_disease_model.h5")
